In [1]:
import scanpy as sc
import pandas as pd
import numpy as np
ad = sc.read_text("exprMatrix.tsv.gz")

In [3]:
meta = pd.read_csv("meta.tsv", sep="\t")
ad.var = meta
print(ad)

AnnData object with n_obs × n_vars = 16774 × 235121
    var: 'V1', 'Cluster', 'Sample', 'Line', 'Protocol', 'Age', 'iPSCorhESC', 'Class', 'State', 'Type', 'Subtype'


First, we find the list of cluster names:

In [5]:
clusterNameSet = set(ad.var['Cluster'])
print('List of Cluster Names:')
print(clusterNameSet)
#print(ad.obs.index)

List of Cluster Names:
{1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41}


Second, we need a way to get the average expression value for a gene for all cells in a cluster:

In [7]:
def getAvgExprForGeneInCluster(gene, cluster):
    exprFrameForCluster = ad[ad.obs.index == gene, ad.var['Cluster'] == cluster].X
    return exprFrameForCluster.mean()

#print(getAvgExprForGeneInCluster('SOX2|SOX2', 15))
#print(ad[ad.obs.index == 'SOX2', ad.var['Cluster_name'] == 'CN3'].X.tolist())

Third, we combine the average expression values for a gene for a cluster in a dataframe:

In [47]:
geneList = ['AQP4', 'SLC1A3', 'HepaCAM1', 'CD44', 'NCAM1', 'CD24', 'FUT4', \
               'CXCR4', 'FOXO4', 'PDGFRA', 'ITGB2', 'TFRC', 'PROM1', 'NKX2-2']

# gene list corrected by hand using getGeneNameVariantList
correctedGeneList = ['AQP4|AQP4', 'SLC1A3|SLC1A3', 'CD44|CD44', 'NCAM1|NCAM1', 'CD24|CD24', \
                     'FUT4|FUT4', 'CXCR4|CXCR4', 'FOXO4|FOXO4', 'PDGFRA|PDGFRA', 'TFRC|TFRC', \
                     'PROM1|PROM1', 'NKX2-2|NKX2-2']

dataFrame = pd.DataFrame(np.array([[getAvgExprForGeneInCluster(gene, cluster) \
                                    for gene in correctedGeneList] for cluster in clusterNameSet]), \
                                    columns = correctedGeneList, index=clusterNameSet)

print(dataFrame)
dataFrame.to_csv('averageExpressionPerCluster.csv')

    AQP4|AQP4  SLC1A3|SLC1A3  CD44|CD44  NCAM1|NCAM1  CD24|CD24  FUT4|FUT4  \
1    0.000000       0.812414   0.122465     1.199391   1.417542   0.000000   
2    0.000000       0.022298   0.008054     0.864200   2.029322   0.000000   
3    0.000000       0.039525   0.017395     0.395865   1.641140   0.000000   
4    0.000000       0.327585   0.008680     0.250124   0.584714   0.000000   
5    0.000000       0.040559   0.014556     0.094008   0.511430   0.000150   
6    0.000000       0.000000   0.017995     0.221289   0.868255   0.000000   
7    0.000000       0.068045   0.011062     0.170758   0.911565   0.000000   
8    0.000000       0.030339   0.000758     0.419827   1.020794   0.000000   
9    0.000000       0.023159   0.019347     0.301115   1.552736   0.000697   
10   0.071869       0.639734   0.000000     0.199962   0.504025   0.000000   
11   0.000000       0.338309   0.054068     0.273511   0.454990   0.000595   
12   0.000000       0.086429   0.021728     0.149900   1.699158 

The following are tools to help with gene name selection for the list of target genes:

The function below returns a list of variants of a gene name found in the dataset. It checks for substrings both ways, in a case insensitive way. One tuple with a match of NONE is returned in the list if no match is found. No attempt is made to filter out gene names that are substrings of one another without being related.

In [48]:
#dataset at https://cells.ucsc.edu/?ds=organoidreportcard contains gene names
#in the form GENENAME|GENENAME and makes the comparison unhelpful
#when searching for the variable as a substring in the target gene.
def getGeneVariantList(targetGene):
    geneVariantList = []
    for variable in ad.obs.index:
        if (targetGene.lower() in variable.lower() \
            or variable.lower() in targetGene.lower()):
            geneVariantList.append((targetGene, variable))
                
    if targetGene not in [v[0] for v in geneVariantList]:
        geneVariantList.append((targetGene, 'NONE'))
        
    return geneVariantList
        
print(getGeneVariantList('HepaCAM1'))
print(getGeneVariantList('HepaCAM'))

[('HepaCAM1', 'NONE')]
[('HepaCAM', 'HEPACAM|HEPACAM')]


In the block below, a dictionary of lists of tuples is returned where genes in the target list are variants of the names in the dataset, as per getGeneVariantList.

In [49]:
geneVariantDict = {}
for targetGene in geneList:
    geneVariantDict[targetGene] = getGeneVariantList(targetGene)

for key in allGeneVariants.keys():
    print(geneVariantDict[key])
    
#print(allGeneVariants)

[('AQP4', 'AQP4|AQP4'), ('AQP4', 'AQP4-AS1|AQP4-AS1')]
[('SLC1A3', 'SLC1A3|SLC1A3')]
[('HepaCAM1', 'NONE')]
[('CD44', 'CD44|CD44')]
[('NCAM1', 'NCAM1|NCAM1')]
[('CD24', 'CD24|CD24'), ('CD24', 'CD248|CD248'), ('CD24', 'CD247|CD247')]
[('FUT4', 'FUT4|FUT4')]
[('CXCR4', 'CXCR4|CXCR4')]
[('FOXO4', 'FOXO4|FOXO4')]
[('PDGFRA', 'PDGFRA|PDGFRA')]
[('ITGB2', 'NONE')]
[('TFRC', 'TFRC|TFRC')]
[('PROM1', 'PROM1|PROM1')]
[('NKX2-2', 'NKX2-2|NKX2-2')]
